In [13]:
import os
import sys
import subprocess
import numpy as np
import nibabel as nib
from megsurfer import *

In [15]:
postprocess_freesurfer_surfaces('sub-104',                                
                                './test_output',
                                'multilayer.11.ds.link_vector.gii',
                                n_surfaces=11, 
                                ds_factor=0.1, 
                                orientation='link', 
                                remove_deep=True)

mris_expand -thickness /home/bonaiuto/Dropbox/Projects/software/MEGsurfer/python/test_data/fs/sub-104/surf/lh.white 0.9 /home/bonaiuto/Dropbox/Projects/software/MEGsurfer/python/test_data/fs/sub-104/surf/lh.0.900
using distance as a % of thickness
expanding surface /home/bonaiuto/Dropbox/Projects/software/MEGsurfer/python/test_data/fs/sub-104/surf/lh.white by 90.0% of thickness and writing it to /home/bonaiuto/Dropbox/Projects/software/MEGsurfer/python/test_data/fs/sub-104/surf/lh.0.900
reading thickness...
-01: dt=0.0000, 0 negative triangles
ending sse = 9707.569352
nrounds = 6
step 432 of 432     
writing expanded surface to /home/bonaiuto/Dropbox/Projects/software/MEGsurfer/python/test_data/fs/sub-104/surf/lh.0.900...
surface expansion took 29 minutes and 29 seconds.
mris_expand -thickness /home/bonaiuto/Dropbox/Projects/software/MEGsurfer/python/test_data/fs/sub-104/surf/rh.white 0.9 /home/bonaiuto/Dropbox/Projects/software/MEGsurfer/python/test_data/fs/sub-104/surf/rh.0.900
using

-01: dt=0.0000, 0 negative triangles
ending sse = 9707.569352
nrounds = 6
step 192 of 192     
writing expanded surface to /home/bonaiuto/Dropbox/Projects/software/MEGsurfer/python/test_data/fs/sub-104/surf/lh.0.400...
surface expansion took 9 minutes and 40 seconds.
mris_expand -thickness /home/bonaiuto/Dropbox/Projects/software/MEGsurfer/python/test_data/fs/sub-104/surf/rh.white 0.3999999999999999 /home/bonaiuto/Dropbox/Projects/software/MEGsurfer/python/test_data/fs/sub-104/surf/rh.0.400
using distance as a % of thickness
expanding surface /home/bonaiuto/Dropbox/Projects/software/MEGsurfer/python/test_data/fs/sub-104/surf/rh.white by 40.0% of thickness and writing it to /home/bonaiuto/Dropbox/Projects/software/MEGsurfer/python/test_data/fs/sub-104/surf/rh.0.400
reading thickness...
-01: dt=0.0000, 0 negative triangles
ending sse = 9976.720387
nrounds = 6
step 192 of 192     
writing expanded surface to /home/bonaiuto/Dropbox/Projects/software/MEGsurfer/python/test_data/fs/sub-104/su